In [ ]:
import numpy as np
import matplotlib
import pydicom
from pydicom import dcmread
from pydicom.data import get_testdata_file
import os
from pathlib import Path
%matplotlib inline

In [ ]:
directory='6'
# fetch the path to the test data
path = get_testdata_file('{}\\DICOMDIR'.format(directory))
ds = dcmread('{}/DICOMDIR'.format(directory))
root_dir = Path(ds.filename).resolve().parent
print(f'Root directory: {root_dir}\n')

In [ ]:
#To list the patients in the record 
ds.patient_records

# to select the only patient in this record
for patient in ds.patient_records:
    patient

# Find all the STUDY records for the patient
studies = [ii for ii in patient.children if ii.DirectoryRecordType == "STUDY"]

# Select the only study in this patient record
for study in studies:
    study

# Find all the SERIES records in the study
all_series = [ii for ii in study.children if ii.DirectoryRecordType == "SERIES"]

In [ ]:
all_series

In [ ]:
#Choose the target series 
selected_series=all_series[7] # 7 is the index of the target series in the all_series list

In [ ]:

# Find all the IMAGE records in the series
images = [ii for ii in selected_series.children if ii.DirectoryRecordType == "IMAGE"]

plural = ('', 's')[len(images) > 1]

descr = getattr(selected_series, "SeriesDescription", "(no value available)")

print(f"{'  ' * 2}SERIES: SeriesNumber={selected_series.SeriesNumber}, "
    f"Modality={selected_series.Modality}, SeriesDescription={descr} - "
    f"{len(images)} SOP Instance{plural}")
# Get the absolute file path to each instance
#   Each IMAGE contains a relative file path to the root directory
elems = [ii["ReferencedFileID"] for ii in images]
# Make sure the relative file path is always a list of str
paths = [[ee.value] if ee.VM == 1 else ee.value for ee in elems]
paths = [Path(*p) for p in paths]

# List the instance file paths
for p in paths:
    print(f"{'  ' * 3}IMAGE: Path={os.fspath(p)}")

In [ ]:

# Get the absolute file path to each instance
#   Each IMAGE contains a relative file path to the root directory
elems = [ii["ReferencedFileID"] for ii in images]
# Make sure the relative file path is always a list of str
paths = [[ee.value] if ee.VM == 1 else ee.value for ee in elems]
paths = [Path(*p) for p in paths]

# List the instance file paths
for p in paths:
    print(f"{'  ' * 3}IMAGE: Path={os.fspath(p)}")

    # Optionally read the corresponding SOP Instance
    # instance = dcmread(Path(root_dir) / p)
    # print(instance.PatientName)

In [ ]:
str_path=[]
for path in paths:
    str_path.append(str(path))

In [ ]:
complete_path=[]
for path in str_path:
    complete_path.append("{}\\".format(directory)+path)

In [ ]:
complete_path

In [ ]:
# Get ref file
RefDs = pydicom.filereader.dcmread(str(complete_path[0]))

# Load dimensions based on the number of rows, columns, and slices (along the Z axis)
ConstPixelDims = (int(RefDs.Rows), int(RefDs.Columns), len(complete_path))

# Load spacing values (in mm)
ConstPixelSpacing = (float(RefDs.PixelSpacing[0]), float(RefDs.PixelSpacing[1]), float(RefDs.SliceThickness))

In [ ]:
# The array is sized based on 'ConstPixelDims'
ArrayDicom = np.zeros(ConstPixelDims , dtype=RefDs.pixel_array.dtype)

# loop through all the DICOM files
for filenameDCM in complete_path:
    # read the file
    ds = pydicom.read_file((filenameDCM))
    # store the raw image data
    ArrayDicom[:, :, complete_path.index(filenameDCM)] = ds.pixel_array

In [ ]:
for s in range(ArrayDicom.shape[2]):
    matplotlib.pyplot.figure(dpi=250)
    matplotlib.pyplot.axes().set_aspect('equal', 'datalim')
    matplotlib.pyplot.set_cmap(matplotlib.pyplot.gray())
    matplotlib.pyplot.pcolormesh(np.flipud(ArrayDicom[:, :, s]))
    matplotlib.pyplot.show()
    print(s)